# Project 2 NLP: Hatespeech Classifier

## Authors:

Adrian Obermühlner & Freja Rasmussen

## Resarch Question:

How do different preprocessing methods (nothing, stop word removal, lemming, stemming,…) affect the result of a hate speech classifier?

## Imports

In [23]:
# Imports
import pandas as pd
import numpy as np
from datasets import Dataset
import torch

# Preprocessing imports
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# Tokenizing
from sklearn.feature_extraction.text import TfidfVectorizer

# Model
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dense, Embedding, Dropout
from tensorflow.keras.models import Sequential

TypeError: 'type' object is not subscriptable

In [24]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: NVIDIA GeForce GTX 1650 Ti


## Data Import


In [2]:
RANDOM_SEED = 42
BINARY_LABEL = "is_hate"
CATEGORIES = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

np.random.seed(RANDOM_SEED)  # set random seed for reproducibility
# Make the labels into hate and no hate as 1 and 0

def binarize_labels(df):
    return (df[CATEGORIES].sum(axis=1) > 0).astype(int)

data_train = pd.read_csv("./data/train/train.csv", index_col=0)
data_train[BINARY_LABEL] = binarize_labels(data_train)

data_test = pd.read_csv("./data/test/test.csv", index_col=0).join(
    pd.read_csv("./data/test_labels/test_labels.csv", index_col=0)
)
data_test.drop(data_test[data_test["toxic"] == -1].index, inplace=True)
data_test[BINARY_LABEL] = binarize_labels(data_test)

In [17]:
data_train.head(10)

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,is_hate
id,,,,,,,,
0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,0
000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,0
000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,0
0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,0
0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,0
00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0,0
0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0,1
00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0,0
00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0,0


In [16]:
data_test.head(10)

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,is_hate
id,,,,,,,,
0001ea8717f6de06,Thank you for understanding. I think very high...,0,0,0,0,0,0,0
000247e83dcc1211,:Dear god this site is horrible.,0,0,0,0,0,0,0
0002f87b16116a7f,"""::: Somebody will invariably try to add Relig...",0,0,0,0,0,0,0
0003e1cccfd5a40a,""" \n\n It says it right there that it IS a typ...",0,0,0,0,0,0,0
00059ace3e3e9a53,""" \n\n == Before adding a new product to the l...",0,0,0,0,0,0,0
000663aff0fffc80,this other one from 1897,0,0,0,0,0,0,0
000689dd34e20979,== Reason for banning throwing == \n\n This ar...,0,0,0,0,0,0,0
000844b52dee5f3f,|blocked]] from editing Wikipedia. |,0,0,0,0,0,0,0
00091c35fa9d0465,"== Arabs are committing genocide in Iraq, but ...",1,0,0,0,0,0,1


In [14]:
# get the distribution of the labels to see if roughly similar for both

is_hate_count_train = data_train['is_hate'].value_counts()
ratio_train = is_hate_count_train/ len(data_train)

is_hate_count_test = data_test['is_hate'].value_counts()
ratio_test = is_hate_count_test/ len(data_test)

print('Ratio of no/is hate for train set: ', ratio_train)
print('Ratio of no/is hate for test set: ', ratio_test)

Ratio of no/is hate for train set:  0    0.898321
1    0.101679
Name: is_hate, dtype: float64
Ratio of no/is hate for test set:  0    0.90242
1    0.09758
Name: is_hate, dtype: float64


## Representation

In [ ]:
# Train and Test set
X_train = data_train["comment_text"]
y_train = data_train["is_hate"]

X_test = data_test["comment_text"]
y_test = data_test["is_hate"]

## Data Preprocessing

**Note**: We would need to make a loop for the different combinations of 
preprocessing (none, only stemming, only lemming, only stop word removal and every combination of this)
Either as coloumns that can be used to iterate over for the model training and validation, or make the preprocessing
and then go further and repeat from beginning.


In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    
    # Lowercase all text    
    text = text.lower()
    
    # Tokenize the text into words
    tokens = word_tokenize(text)
    
    # Remove stopwords and apply stemming
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    
    # Lemming of words
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered_tokens if word.isalpha()]
    
    # Join the stemmed words back into a sentence
    return ' '.join(lemmatized_tokens)

data_train['comment_text_clean'] = data_train['comment_text'].apply(preprocess_text)


## Word Embedding

**Note**: How is word embedding implemented? Or is it better to use TF-IDF?

In [ ]:
# Tokenizing with TF-IDF or alternatively
# Look at example of word embedding with cnn for pytorch or tensorflow
# fast text für word embedding from facebook, maybe shorten down to only words in data set, don't forget to add padding token
# Tokenize the comment, hand the vector with padding(for longest comment in batch so all have same size) to CNN
# in CNN first layer is embedding, this layer genrates matrix with size vocab dimensions x length of longest comment
# Then use CNN as desired for output with softmax at the end for the classification


## Model Implementation

**Note**: Does a CNN makes sense for sentiment analysis? or a simpler model?

In [ ]:
# Make a CNN with PyTorch using skorch as wrapper to make it possible to use sklearn.pipeline with the model
# This way gridsearch for hyper parameters is possible and tfidfVectorizer can be used for tf-idf
# CNN: vector size 300, conv. layer of some size, flatten, relu, end with softmax or something
# Example: https://www.kaggle.com/code/raviusz/jigsaw-toxic-comment

## Test with Testset

**Note**: We will use the given test set to compare the different approaches. Make a dataframe with all the results
in accuracy, f1, recall, etc. 

**Note**: Setfit as example of what to use if I do not take CNN

In [ ]:
# set fit
from setfit import SetFitModel, Trainer, TrainingArguments

dataset_train = Dataset.from_pandas(
    # take 10 samples from each class
)
dataset_test = Dataset.from_pandas(
    # take 100 samples from each class
    data_test.groupby("is_hate").sample(100, random_state=RANDOM_SEED)
)

model = SetFitModel.from_pretrained(
    "sentence-transformers/all-MiniLM-L6-v2",
    labels=[0, 1],
).to("cpu")

args = TrainingArguments(
    batch_size=1,
    num_epochs=1,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    num_iterations=20,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset_train,
    eval_dataset=dataset_test,
    metric="accuracy",
    column_mapping={"comment_text": "text", "is_hate": "label"}
)
trainer.train()
trainer.evaluate()

y_true = dataset_test["is_hate"]
y_pred_setfit = model.predict(dataset_test["comment_text"])
print(classification_report(y_true, y_pred_setfit))